**CODE FOR SEMG DATASET GENERATION**

In [1]:
# This code loads the sEMG files from the "Database" folder, and converts them into spectrogram images.
# Each trial is named: grasp_type + channel number + trial number. E.g. cyl_ch1_1.csv
# Each trial contains 2 images: cyl_ch1_1.csv and cyl_ch2_1.csv are converted to individual images.

In [2]:
import numpy as np
from scipy.signal import stft
import matplotlib.pyplot as plt
from scipy.signal import spectrogram
from PIL import Image

In [ ]:
# Two versions of spectrograms were computed: one with squared values, and one without.
# The squared values approach helped emphasizing the differences in the value range of the spectrogram.

# Both versions can be used for the classifier:
# - Non-squared version is saved in "spectrograms" folder
# - Squaed version is saved in "spectrograms_squared" folder

**NON_SQUARED VERSION OF SPECTROGRAMS**

In [ ]:
# Definition of 6 grasp types
grasp_type = ['cyl', 'hook', 'lat', 'palm', 'spher', 'tip']

# For loop over each file in Database (each corresponding to a patient)
for grasp in grasp_type:
    for i in range(1,9):

        # File selection for a patient, with channels 1 and 2 individually
        data_ch1 = np.loadtxt('./Database/'+grasp + '_ch1_'+str(i)+'.csv', delimiter=',')
        data_ch2 = np.loadtxt('./Database/'+grasp + '_ch2_'+str(i)+'.csv', delimiter=',')

        # For loop over the trials of each patient and its sEMG signal
        for j in range(data_ch1.shape[0]):

            # Spectrogram calculation for channel 1 figures
            f, t, Sxx = stft(data_ch1[j,0:2500], fs = 500)
            Sxx = np.abs(Sxx)
            Sxx_scaled = (Sxx - np.min(Sxx)) / (np.max(Sxx) - np.min(Sxx))

            # Appearance modifications for spectrogram figures
            plt.pcolormesh(t, f, Sxx_scaled,shading='gouraud')
            plt.ylabel('Frequency [Hz]')
            plt.xlabel('Time [sec]')
            plt.axis('off')
            plt.set_cmap('jet')
            #plt.show()
            plt.savefig('./semg_dataset/spectrograms/'+grasp + '_ch1_'+str(i)+'_'+str(j)+'.png')
            plt.close()
            
            # Spectrogram calculation for channel 2 figures
            f, t, Sxx = stft(data_ch2[j,0:2500], fs = 500)
            Sxx = np.abs(Sxx)
            Sxx_scaled = (Sxx - np.min(Sxx)) / (np.max(Sxx) - np.min(Sxx))

            # Appearance modifications for spectrogram figures
            plt.pcolormesh(t, f, Sxx_scaled,shading='gouraud')
            plt.ylabel('Frequency [Hz]')
            plt.xlabel('Time [sec]')
            plt.axis('off')
            plt.set_cmap('jet')
            #plt.show()
            plt.savefig('./semg_dataset/spectrograms/'+grasp + '_ch2_'+str(i)+'_'+str(j)+'.png')
            plt.close()
            
            # Control print
            print('Figure ' + str(j) + 'from subject ' + str(i) + 'saved')

**SQUARED VERSION OF SPECTROGRAMS**

In [ ]:
# Definition of 6 grasp types
grasp_type = ['cyl', 'hook', 'lat', 'palm', 'spher', 'tip']

# For loop over each file in Database (each corresponding to a patient)
for grasp in grasp_type:
    for i in range(1,9):

         # File selection for a patient, with channels 1 and 2 individually       
        data_ch1 = np.loadtxt('./Database/'+grasp + '_ch1_'+str(i)+'.csv', delimiter=',')
        data_ch2 = np.loadtxt('./Database/'+grasp + '_ch2_'+str(i)+'.csv', delimiter=',')
        
         # For loop over the trials of each patient and its sEMG signal
        for j in range(data_ch1.shape[0]):

            # Spectrogram calculation for channel 1 figures
            f, t, Sxx = stft(data_ch1[j,0:2500], fs = 500)
            Sxx = np.abs(Sxx)**2 # main difference
            Sxx_scaled = (Sxx - np.min(Sxx)) / (np.max(Sxx) - np.min(Sxx))
            
            # Appearance modifications for spectrogram figures
            plt.pcolormesh(t, f, Sxx_scaled,shading='gouraud')
            plt.ylabel('Frequency [Hz]')
            plt.xlabel('Time [sec]')
            plt.axis('off')
            plt.set_cmap('jet')
            #plt.show()
            plt.savefig('./semg_dataset/spectrograms_squared/'+grasp + '_ch1_'+str(i)+'_'+str(j)+'.png')
            plt.close()

            # Spectrogram calculation for channel 2 figures
            f, t, Sxx = stft(data_ch2[j,0:2500], fs = 500)
            Sxx = np.abs(Sxx)**2
            Sxx_scaled = (Sxx - np.min(Sxx)) / (np.max(Sxx) - np.min(Sxx))
            
            # Appearance modifications for spectrogram figures
            plt.pcolormesh(t, f, Sxx_scaled,shading='gouraud')
            plt.ylabel('Frequency [Hz]')
            plt.xlabel('Time [sec]')
            plt.axis('off')
            plt.set_cmap('jet')
            #plt.show()
            plt.savefig('./semg_dataset/spectrograms_squared/'+grasp + '_ch2_'+str(i)+'_'+str(j)+'.png')
            plt.close()

             # Control print
            print('Figure ' + str(j) + 'from subject ' + str(i) + 'saved')

**STACK OF SPECTROGRAMS**

In [ ]:
# This part of the code generates the final dataset
# For each trial, channel 1 and channel 2 RGB images are stacked into a 6-D numpy volume, to be fed into the classifier

# The results are saved in "spectrograms_concat"

In [ ]:
# Definition of grasp types
grasp_type = ['cyl', 'hook', 'lat', 'palm', 'spher', 'tip']

# Loop over the first 6 patients, who have 30 trials each
for grasp in grasp_type:
    for i in range(1,6):
        for j in range(30):

            # Load two spectrograms
            img1 = Image.open('./semg_dataset/spectrograms/'+grasp + '/'+ grasp + '_ch1_'+str(i)+'_'+str(j)+'.png')
            img2 = Image.open('./semg_dataset/spectrograms/'+grasp + '/'+ grasp + '_ch2_'+str(i)+'_'+str(j)+'.png')
            
            # Set as numpy arrays
            np1 = np.asarray(img1)[:,:,:3]
            np2 = np.asarray(img2)[:,:,:3]
            
            #img1.show()
            #img2.show()
            
            # Stack into 6D array
            np_out = np.concatenate((np1,np2),2)
            
            # Save the results
            print('File ' + str(30*(i-1)+(j+1)) + ' saved')
            np.save('./semg_dataset/spectrograms_concat/'+grasp + '/'+ grasp +'_'+ str(30*(i-1)+(j+1)), np_out)


# Loop over the last 4 patients, who have 100 trials each
for grasp in grasp_type:
    for i in range(1,4):
        for j in range(100):

            # Load two spectrograms
            img1 = Image.open('./semg_dataset/spectrograms/'+grasp + '/'+ grasp + '_ch1_'+str(i+5)+'_'+str(j)+'.png')
            img2 = Image.open('./semg_dataset/spectrograms/'+grasp + '/'+ grasp + '_ch2_'+str(i+5)+'_'+str(j)+'.png')

            # Set as numpy arrays
            np1 = np.asarray(img1)[:,:,:3]
            np2 = np.asarray(img2)[:,:,:3]
            
            #img1.show()
            #img2.show()
            
            # Stack into 6D array
            np_out = np.concatenate((np1,np2),2)
            
            # Save the results
            print('File ' + str(150+100*(i-1)+(j+1)) + ' saved')
            np.save('./semg_dataset/spectrograms_concat/'+grasp + '/'+ grasp +'_'+ str(150+100*(i-1)+(j+1)), np_out)

**SEMG DATASET IS READY AT THIS POINT** - FOUND IN "SPECTROGRAMS_CONCAT"